In [ ]:
import math

import torch

from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module


class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GCN_DECONF(nn.Module):
    def __init__(self, nfeat, nhid, dropout, n_in=1, n_out=1, cuda=False):
        super(GCN_DECONF, self).__init__()


        # self.gc2 = GraphConvolution(nhid, nclass)

        if cuda:
            self.gc = [GraphConvolution(nfeat, nhid).cuda()]
            for i in range(n_in - 1):
                self.gc.append(GraphConvolution(nhid, nhid).cuda())
        else:
            self.gc = [GraphConvolution(nfeat, nhid)]
            for i in range(n_in - 1):
                self.gc.append(GraphConvolution(nhid, nhid))


        print(self.gc)

        self.n_in = n_in
        self.n_out = n_out

        if cuda:

            self.out_t00 = [nn.Linear(nhid,nhid).cuda() for i in range(n_out)]
            self.out_t10 = [nn.Linear(nhid,nhid).cuda() for i in range(n_out)]
            self.out_t01 = nn.Linear(nhid,1).cuda()
            self.out_t11 = nn.Linear(nhid,1).cuda()

        else:
            self.out_t00 = [nn.Linear(nhid,nhid) for i in range(n_out)]
            self.out_t10 = [nn.Linear(nhid,nhid) for i in range(n_out)]
            self.out_t01 = nn.Linear(nhid,1)
            self.out_t11 = nn.Linear(nhid,1)

        self.dropout = dropout

        # a linear layer for propensity prediction
        self.pp = nn.Linear(nhid, 1)

        if cuda:
            self.pp = self.pp.cuda()
        self.pp_act = nn.Sigmoid()

    def forward(self, x, adj, t, cuda=False):

        rep = F.relu(self.gc[0](x, adj))
        rep = F.dropout(rep, self.dropout, training=self.training)
        for i in range(1, self.n_in):
            rep = F.relu(self.gc[i](rep, adj))
            rep = F.dropout(rep, self.dropout, training=self.training)

        for i in range(self.n_out):

            y00 = F.relu(self.out_t00[i](rep))
            y00 = F.dropout(y00, self.dropout, training=self.training)
            y10 = F.relu(self.out_t10[i](rep))
            y10 = F.dropout(y10, self.dropout, training=self.training)

        y0 = self.out_t01(y00).view(-1)
        y1 = self.out_t11(y10).view(-1)

        # print(t.shape,y1.shape,y0.shape)
        y = torch.where(t > 0,y1,y0)

        p1 = self.pp_act(self.pp(rep)).view(-1)

        return y, rep, p1

In [ ]:
import numpy as np
import scipy.io as sio
import scipy.sparse as sp

import torch
import torch.nn.functional as F

import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc


def sparse_mx_to_torch_sparse_tensor(sparse_mx,cuda=False):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)

    sparse_tensor = torch.sparse.FloatTensor(indices, values, shape)
    if cuda:
        sparse_tensor = sparse_tensor.cuda()
    return sparse_tensor

def normalize(mx):
	"""Row-normalize sparse matrix"""
	rowsum = np.array(mx.sum(1))
	r_inv = np.power(rowsum, -1).flatten()
	r_inv[np.isinf(r_inv)] = 0.
	r_mat_inv = sp.diags(r_inv)
	mx = r_mat_inv.dot(mx)
	return mx

def load_data(path, name='BlogCatalog',exp_id='0',original_X = False, extra_str=""):
	data = sio.loadmat(path+name+extra_str+'/'+name+exp_id+'.mat')
	A = data['Network'] #csr matrix

	# try:
	# 	A = np.array(A.todense())
	# except:
	# 	pass

	if not original_X:
		X = data['X_100']
	else:
		X = data['Attributes']

	Y1 = data['Y1']
	Y0 = data['Y0']
	T = data['T']

	return X, A, T, Y1, Y0


def wasserstein(x,y,p=0.5,lam=10,its=10,sq=False,backpropT=False,cuda=False):
    """return W dist between x and y"""
    '''distance matrix M'''
    nx = x.shape[0]
    ny = y.shape[0]

    x = x.squeeze()
    y = y.squeeze()

#    pdist = torch.nn.PairwiseDistance(p=2)

    M = pdist(x,y) #distance_matrix(x,y,p=2)

    '''estimate lambda and delta'''
    M_mean = torch.mean(M)
    M_drop = F.dropout(M,10.0/(nx*ny))
    delta = torch.max(M_drop).detach()
    eff_lam = (lam/M_mean).detach()

    '''compute new distance matrix'''
    Mt = M
    # Explicitly create tensors on the correct device
    row = delta*torch.ones(M[0:1,:].shape, device=x.device)
    col = torch.cat([delta*torch.ones(M[:,0:1].shape, device=x.device),torch.zeros((1,1), device=x.device)],0)
    if cuda:
        row = row.cuda()
        col = col.cuda()
    Mt = torch.cat([M,row],0)
    Mt = torch.cat([Mt,col],1)

    '''compute marginal'''
    a = torch.cat([p*torch.ones((nx,1), device=x.device)/nx,(1-p)*torch.ones((1,1), device=x.device)],0)
    b = torch.cat([(1-p)*torch.ones((ny,1), device=x.device)/ny, p*torch.ones((1,1), device=x.device)],0)

    '''compute kernel'''
    Mlam = eff_lam * Mt
    temp_term = torch.ones(1, device=x.device)*1e-6
    if cuda:
        temp_term = temp_term.cuda()
        a = a.cuda()
        b = b.cuda()
    K = torch.exp(-Mlam) + temp_term
    U = K * Mt
    ainvK = K/a

    u = a

    for i in range(its):
        u = 1.0/(ainvK.matmul(b/torch.t(torch.t(u).matmul(K))))
        if cuda:
            u = u.cuda()
    v = b/(torch.t(torch.t(u).matmul(K)))
    if cuda:
        v = v.cuda()

    upper_t = u*(torch.t(v)*K).detach()

    E = upper_t*Mt
    D = 2*torch.sum(E)

    if cuda:
        D = D.cuda()

    return D, Mlam

def pdist(sample_1, sample_2, norm=2, eps=1e-5):
    """Compute the matrix of all squared pairwise distances.
    Arguments
    ---------
    sample_1 : torch.Tensor or Variable
        The first sample, should be of shape ``(n_1, d)``.
    sample_2 : torch.Tensor or Variable
        The second sample, should be of shape ``(n_2, d)``.
    norm : float
        The l_p norm to be used.
    Returns
    -------
    torch.Tensor or Variable
        Matrix of shape (n_1, n_2). The [i, j]-th entry is equal to
        ``|| sample_1[i, :] - sample_2[j, :] ||_p``."""
    n_1, n_2 = sample_1.size(0), sample_2.size(0)
    norm = float(norm)
    if norm == 2.:
        norms_1 = torch.sum(sample_1**2, dim=1, keepdim=True)
        norms_2 = torch.sum(sample_2**2, dim=1, keepdim=True)
        norms = (norms_1.expand(n_1, n_2) +
                 norms_2.transpose(0, 1).expand(n_1, n_2))
        distances_squared = norms - 2 * sample_1.mm(sample_2.t())
        return torch.sqrt(eps + torch.abs(distances_squared))
    else:
        dim = sample_1.size(1)
        expanded_1 = sample_1.unsqueeze(1).expand(n_1, n_2, dim)
        expanded_2 = sample_2.unsqueeze(0).expand(n_1, n_2, dim)
        differences = torch.abs(expanded_1 - expanded_2) ** norm
        inner = torch.sum(differences, dim=2, keepdim=False)
        return (eps + inner) ** (1. / norm)

# def sklearn_auc_score(t,ps):
#     """
#
#     :param t: observed treatment (ground truth)
#     :param ps: propensity score
#     :return: auc score
#     """
#
#     # Compute ROC curve and ROC area for each class
#     fpr = dict()
#     tpr = dict()
#     roc_auc = dict()
#     for i in range(n_classes):
#         fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
#         roc_auc[i] = auc(fpr[i], tpr[i])
#
#     # Compute micro-average ROC curve and ROC area
#     fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
#     roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
#
#     plt.figure()
#     lw = 2
#     plt.plot(fpr[2], tpr[2], color='darkorange',
#              lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
#     plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
#     plt.xlim([0.0, 1.0])
#     plt.ylim([0.0, 1.05])
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.title('Receiver operating characteristic example')
#     plt.legend(loc="lower right")
    # plt.show()
    # plt.savefig('./figs/' + name + extra_str + str(exp_id) + 'ps_dist.pdf', bbox_inches='tight')

#def distance_matrix(x,y,p=2):
#    """ Computes the squared Euclidean distance between all pairs x in X, y in Y """
#    x = x.squeeze()
#    y = y.squeeze()
#    C = -2*x.matmul(torch.t(y))
#    nx = torch.sum(x.pow(2),dim=1).view(-1,1)
#    ny = torch.sum(y.pow(2),dim=1).view(-1,1)
#    D = (C + torch.t(ny)) + nx
#    return D


In [ ]:
import time
import argparse
import numpy as np

import torch
# import torch.nn.functional as F
import torch.optim as optim


# from scipy import sparse as sp
import csv

# Training settings
parser = argparse.ArgumentParser()
parser.add_argument('--nocuda', type=int, default=0,
                    help='Disables CUDA training.')
parser.add_argument('--dataset', type=str, default='BlogCatalog')
parser.add_argument('--extrastr', type=str, default='_random') # 0.5,1,2, _random

parser.add_argument('--seed', type=int, default=42, help='Random seed.')
parser.add_argument('--epochs', type=int, default=200,
                    help='Number of epochs to train.')
parser.add_argument('--lr', type=float, default=1e-2,
                    help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=1e-5,
                    help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=100,
                    help='Number of hidden units.')
parser.add_argument('--dropout', type=float, default=0.1,
                    help='Dropout rate (1 - keep probability).')
parser.add_argument('--alpha', type=float, default=1e-4,
                    help='trade-off of representation balancing.')
parser.add_argument('--clip', type=float, default=100.,
                    help='gradient clipping')
parser.add_argument('--nout', type=int, default=2)
parser.add_argument('--nin', type=int, default=2)

parser.add_argument('--tr', type=float, default=0.6)
parser.add_argument(
    '--path', type=str, default='/content/drive/MyDrive/Causal_network_matching/data/')
parser.add_argument('--normy', type=int, default=1)

args = parser.parse_args(args=[])
args.cuda = not args.nocuda and torch.cuda.is_available()
Tensor = torch.cuda.FloatTensor if args.cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if args.cuda else torch.LongTensor

alpha = Tensor([args.alpha])

np.random.seed(args.seed)
torch.manual_seed(args.seed)

loss = torch.nn.MSELoss()
bce_loss = torch.nn.BCEWithLogitsLoss()

if args.cuda:
    torch.cuda.manual_seed(args.seed)
    alpha = alpha.cuda()
    loss = loss.cuda()
    bce_loss = bce_loss.cuda()

def prepare(i_exp):

    # Load data and init models
    X, A, T, Y1, Y0 = load_data(args.path, name=args.dataset, original_X=False, exp_id=str(i_exp), extra_str=args.extrastr)

    n = X.shape[0]
    n_train = int(n * args.tr)
    n_test = int(n * 0.2)
    # n_valid = n_test

    idx = np.random.permutation(n)
    idx_train, idx_test, idx_val = idx[:n_train], idx[n_train:n_train+n_test], idx[n_train+n_test:]

    X = normalize(X) #row-normalize
    # A = utils.normalize(A+sp.eye(n))

    X = X.todense()
    X = Tensor(X)

    Y1 = Tensor(np.squeeze(Y1))
    Y0 = Tensor(np.squeeze(Y0))
    T = LongTensor(np.squeeze(T))

    A = sparse_mx_to_torch_sparse_tensor(A,cuda=args.cuda)

    # print(X.shape, Y1.shape, A.shape)

    idx_train = LongTensor(idx_train)
    idx_val = LongTensor(idx_val)
    idx_test = LongTensor(idx_test)

    # Model and optimizer
    model = GCN_DECONF(nfeat=X.shape[1],
                nhid=args.hidden,
                dropout=args.dropout,n_out=args.nout,n_in=args.nin,cuda=args.cuda)

    optimizer = optim.Adam(model.parameters(),
                           lr=args.lr, weight_decay=args.weight_decay)

    return X, A, T, Y1, Y0, idx_train, idx_val, idx_test, model, optimizer


def train(epoch, X, A, T, Y1, Y0, idx_train, idx_val, model, optimizer):
    t = time.time()
    model.train()
#    torch.nn.utils.clip_grad_norm(model.parameters(), args.clip)
    optimizer.zero_grad()
    yf_pred, rep, p1 = model(X, A, T)
    ycf_pred, _, p1 = model(X, A, 1-T)

    # representation balancing, you can try different distance metrics such as MMD
    rep_t1, rep_t0 = rep[idx_train][(T[idx_train] > 0).nonzero()], rep[idx_train][(T[idx_train] < 1).nonzero()]
    dist, _ = wasserstein(rep_t1, rep_t0, cuda=args.cuda)

    YF = torch.where(T>0,Y1,Y0)
    # YCF = torch.where(T>0,Y0,Y1)

    if args.normy:
        # recover the normalized outcomes
        ym, ys = torch.mean(YF[idx_train]), torch.std(YF[idx_train])
        YFtr, YFva = (YF[idx_train] - ym) / ys, (YF[idx_val] - ym) / ys
    else:
        YFtr = YF[idx_train]
        YFva = YF[idx_val]

    loss_train = loss(yf_pred[idx_train], YFtr) + alpha * dist

    # acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    if epoch%10==0:
        # validation
        loss_val = loss(yf_pred[idx_val], YFva) + alpha * dist

        y1_pred, y0_pred = torch.where(T>0,yf_pred,ycf_pred), torch.where(T>0,ycf_pred,yf_pred)
        # Y1, Y0 = torch.where(T>0, YF, YCF), torch.where(T>0, YCF, YF)
        if args.normy:
            y1_pred, y0_pred = y1_pred * ys + ym, y0_pred * ys + ym

        # in fact, you are not supposed to do model selection w. pehe and mae_ate
        # but it is possible to calculate with ITE ground truth (which often isn't available)

        # pehe_val = torch.sqrt(loss((y1_pred - y0_pred)[idx_val],(Y1 - Y0)[idx_val]))
        # mae_ate_val = torch.abs(
        #     torch.mean((y1_pred - y0_pred)[idx_val])-torch.mean((Y1 - Y0)[idx_val]))

        print('Epoch: {:04d}'.format(epoch+1),
              'loss_train: {:.4f}'.format(loss_train.item()),
              'loss_val: {:.4f}'.format(loss_val.item()),
              # 'pehe_val: {:.4f}'.format(pehe_val.item()),
              # 'mae_ate_val: {:.4f}'.format(mae_ate_val.item()),
              'time: {:.4f}s'.format(time.time() - t))

def eva(X, A, T, Y1, Y0, idx_train, idx_test, model, i_exp):
    model.eval()
    yf_pred, rep, p1 = model(X, A, T) # p1 can be used as propensity scores
    # yf = torch.where(T>0, Y1, Y0)
    ycf_pred, _, _ = model(X, A, 1-T)

    YF = torch.where(T>0,Y1,Y0)
    YCF = torch.where(T>0,Y0,Y1)

    ym, ys = torch.mean(YF[idx_train]), torch.std(YF[idx_train])
    # YFtr, YFva = (YF[idx_train] - ym) / ys, (YF[idx_val] - ym) / ys

    y1_pred, y0_pred = torch.where(T>0,yf_pred,ycf_pred), torch.where(T>0,ycf_pred,yf_pred)

    if args.normy:
        y1_pred, y0_pred = y1_pred * ys + ym, y0_pred * ys + ym

    # Y1, Y0 = torch.where(T>0, YF, YCF), torch.where(T>0, YCF, YF)
    pehe_out = torch.sqrt(loss((y1_pred - y0_pred)[idx_test],(Y1 - Y0)[idx_test]))
    ATE_error_out = torch.abs(torch.mean((y1_pred - y0_pred)[idx_test])-torch.mean((Y1 - Y0)[idx_test]))





    idx_train_val = torch.cat([idx_train,idx_val])

    T_numpy = T.detach().cpu().numpy()

    y1_pred[np.where(T_numpy == 1)] = Y1[np.where(T_numpy == 1)]

    y0_pred[np.where(T_numpy == 0)] = Y0[np.where(T_numpy == 0)]



    pehe_in = torch.sqrt(loss((y1_pred - y0_pred)[idx_train_val],(Y1 - Y0)[idx_train_val]))
    ATE_error_in = torch.abs(torch.mean((y1_pred - y0_pred)[idx_train_val])-torch.mean((Y1 - Y0)[idx_train_val]))


    pehe_out = pehe_out.item()
    ATE_error_out = ATE_error_out.item()

    pehe_in = pehe_in.item()
    ATE_error_in = ATE_error_in.item()


    return ATE_error_in , pehe_in, ATE_error_out, pehe_out


if __name__ == '__main__':

    ATE_error_in_list = []
    pehe_in_list = []

    ATE_error_out_list = []
    pehe_out_list = []

    import time
    # Record the start time
    start_time = time.time()

    for i_exp in range(0,10):

        # Train model
        X, A, T, Y1, Y0, idx_train, idx_val, idx_test, model, optimizer = prepare(i_exp)
        t_total = time.time()
        for epoch in range(args.epochs):
            train(epoch, X, A, T, Y1, Y0, idx_train, idx_val, model, optimizer)
        print("Optimization Finished!")
        print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

        # Testing
        errors = eva(X, A, T, Y1, Y0, idx_train, idx_test, model, i_exp)

        ATE_error_in_list.append(errors[0])
        pehe_in_list.append(errors[1])

        ATE_error_out_list.append(errors[2])
        pehe_out_list.append(errors[3])

    end_time = time.time()
    elapsed_time = (end_time - start_time)/10

    print("In-sample ATE absolute error for NetDeconf =", round(np.mean(ATE_error_in_list),2),"+-", round((np.std(ATE_error_in_list, ddof=1) / np.sqrt(np.size(ATE_error_in_list))),2))
    print("In-sample PEHE RMSE for NetDeconf =", round(np.mean(pehe_in_list),2),"+-", round((np.std(pehe_in_list, ddof=1) / np.sqrt(np.size(pehe_in_list))),2))

    print("Out-sample ATE error for NetDeconf =", round(np.mean(ATE_error_out_list),2),"+-", round((np.std(ATE_error_out_list, ddof=1) / np.sqrt(np.size(ATE_error_out_list))),2))
    print("Out-sample PEHE error for NetDeconf =", round(np.mean(pehe_out_list),2),"+-", round((np.std(pehe_out_list, ddof=1) / np.sqrt(np.size(pehe_out_list))),2))

    print(" Wall time in seconds for NetDeconf for 1 simulation(avg over 10) =", elapsed_time)

<ipython-input-3-af30ea9765b9>:23: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:641.)
  sparse_tensor = torch.sparse.FloatTensor(indices, values, shape)


[GraphConvolution (2195 -> 100), GraphConvolution (100 -> 100)]
Epoch: 0001 loss_train: 4.5342 loss_val: 4.5293 time: 0.7177s
Epoch: 0011 loss_train: 2.0742 loss_val: 1.8407 time: 0.3189s
Epoch: 0021 loss_train: 1.4387 loss_val: 1.2785 time: 0.4158s
Epoch: 0031 loss_train: 0.9391 loss_val: 0.8462 time: 0.4434s
Epoch: 0041 loss_train: 0.7870 loss_val: 0.7256 time: 0.3919s
Epoch: 0051 loss_train: 0.6108 loss_val: 0.6415 time: 0.4530s
Epoch: 0061 loss_train: 0.5266 loss_val: 0.5641 time: 0.3684s
Epoch: 0071 loss_train: 0.4560 loss_val: 0.4880 time: 0.3276s
Epoch: 0081 loss_train: 0.4346 loss_val: 0.4275 time: 0.3796s
Epoch: 0091 loss_train: 0.3661 loss_val: 0.3919 time: 0.3772s
Epoch: 0101 loss_train: 0.3434 loss_val: 0.3540 time: 0.3340s
Epoch: 0111 loss_train: 0.3334 loss_val: 0.3316 time: 0.3545s
Epoch: 0121 loss_train: 0.3140 loss_val: 0.3484 time: 0.4154s
Epoch: 0131 loss_train: 0.2906 loss_val: 0.3032 time: 0.3050s
Epoch: 0141 loss_train: 0.2891 loss_val: 0.2923 time: 0.3202s
Epoch: